In [1]:
import pandas as pd
from itertools import combinations
from pandas.tseries.offsets import Day

pd.set_option('display.max_columns', None)

#nrows = 10000000

# Load bond data
#df1 = pd.read_csv('/Users/jerontan/Desktop/y3s1/fixed income/BondDailyPublic.csv', nrows=nrows)
#df2_raw = pd.read_csv('/Users/jerontan/Desktop/y3s1/fixed income/BondDailyDataPublic.csv', nrows=nrows)
df1 = pd.read_csv('/Users/jerontan/Desktop/y3s1/fixed income/BondDailyPublic.csv')
df2_raw = pd.read_csv('/Users/jerontan/Desktop/y3s1/fixed income/BondDailyDataPublic.csv')

# Ensure df2 has only one row per cusip_id by dropping duplicates (to prevent cartesian product when doing left join)
df2 = df2_raw.drop_duplicates(subset=['cusip_id'], keep='first')

# Add issuer_id as the first 6 characters of cusip_id for corporate bond identification
df1['issuer_id'] = df1['cusip_id'].str[:6]

# Merge the dataframes based on 'cusip_id'
df = pd.merge(df1, df2[['cusip_id', 'maturity']], on=['cusip_id'], how='left')

# Remove cs outliers (e.g., -1% to 10%)
df = df[(df['cs'] >= -0.01) & (df['cs'] <= 0.15)]

# Ensure 'cusip_id', 'cs', 'maturity', 'prclean', and 'trd_exctn_dt' are not missing
df = df.dropna(subset=['cusip_id', 'cs', 'maturity', 'prclean', 'trd_exctn_dt', 'ytm'])

# Convert 'maturity' and 'trd_exctn_dt' to datetime and normalize dates to remove time component
df['trd_exctn_dt'] = pd.to_datetime(df['trd_exctn_dt'], errors='coerce').dt.normalize()
df['maturity'] = pd.to_datetime(df['maturity'], errors='coerce').dt.normalize()



print(f"Number of rows after cleaning: {len(df)}")


# Set parameters
min_trade_days_per_bond = 365  # Minimum number of rows required per cusip_id
min_bonds_per_issuer = 5    # Require data from at least 3 different bonds (cusip_id) for the issuer
min_overlap_days = min_trade_days_per_bond         # Minimum number of overlapping days


# Group by 'issuer_id' and 'cusip_id' (each cusip_id represents a unique bond)
issuer_grouped = df.groupby(['issuer_id', 'cusip_id'])

# Filter to keep only bonds (cusip_id) with at least 'min_trade_days_per_bond' trade days
valid_bonds = issuer_grouped.filter(lambda group: len(group) >= min_trade_days_per_bond)

# After filtering bonds, group by 'issuer_id' to check the number of unique bonds for each issuer
issuer_bond_counts = valid_bonds.groupby('issuer_id')['cusip_id'].nunique()

# Keep only issuers with at least 'min_bonds_per_issuer' unique bonds
valid_issuers = issuer_bond_counts[issuer_bond_counts >= min_bonds_per_issuer].index

# Filter the DataFrame to only include rows for valid issuers
filtered_issuer_df = valid_bonds[valid_bonds['issuer_id'].isin(valid_issuers)]

# Now check for the date range overlap of at least 30 days
def has_date_overlap(group, min_overlap_days):
    # Get the min and max trade dates for each cusip_id
    bond_date_ranges = group.groupby('cusip_id').agg(min_date=('trd_exctn_dt', 'min'), 
                                                     max_date=('trd_exctn_dt', 'max')).reset_index()

    # Create a list to store the date ranges
    date_ranges = list(zip(bond_date_ranges['min_date'], bond_date_ranges['max_date']))

    # Check if there is at least a 30-day overlap between any two bonds
    for i, (min_i, max_i) in enumerate(date_ranges):
        for j, (min_j, max_j) in enumerate(date_ranges):
            if i != j:
                # Calculate the overlap period between the two date ranges
                overlap_start = max(min_i, min_j)
                overlap_end = min(max_i, max_j)
                overlap_days = (overlap_end - overlap_start).days
                if overlap_days >= min_overlap_days:
                    return True
    return False

# Apply the date overlap filter for each issuer group
filtered_with_overlap = filtered_issuer_df.groupby('issuer_id').filter(lambda group: has_date_overlap(group, min_overlap_days))

# Output the final trimmed DataFrame
print(filtered_with_overlap)

# Additional print statements for debugging or validation
print(f"Number of rows in the filtered dataframe: {len(filtered_with_overlap)}")
print(f"Number of unique issuers in the filtered dataframe: {filtered_with_overlap['issuer_id'].nunique()}")
# Output the trimmed DataFrame
#print(filtered_issuer_df)




Number of rows after cleaning: 20485181
          Unnamed: 0   cusip_id trd_exctn_dt    prclean     prfull   acclast  \
578              578  001765AC0   2002-07-01  84.357102  87.057102  2.700000   
579              579  001765AC0   2002-07-02  85.847800  88.597800  2.750000   
580              580  001765AC0   2002-07-08  85.183698  88.058698  2.875000   
581              581  001765AC0   2002-07-10  86.250001  89.175001  2.925000   
582              582  001765AC0   2002-07-12  84.145802  87.170802  3.025000   
...              ...        ...          ...        ...        ...       ...   
21331657    21331657  91159HHZ6   2022-12-23  92.945199  93.122422  0.177222   
21331658    21331658  91159HHZ6   2022-12-27  92.904400  93.085650  0.181250   
21331659    21331659  91159HHZ6   2022-12-28  92.883700  93.068977  0.185278   
21331660    21331660  91159HHZ6   2022-12-29  92.945702  93.143063  0.197361   
21331661    21331661  91159HHZ6   2022-12-30  92.917099  93.118488  0.201389   


## Documentation for cell above: Data Cleaning

Steps
1. Drop duplicates and missing values for key variables
2. Create 'issuer_id' first 6 char of 'cusip_id'
3. Remove outliers for credit spread (remove negative and >10%)
4. Filter 'issuer_id' to only keep issuers with sufficient data for trading
    - each bond of the issuer must have at least 365 trading days of overlap with bonds of at least 4 other distinct maturities 
    - so that curvature comparisons can be made within the same time frame

In [2]:
df = filtered_with_overlap

## Create maturity buckets for simplicity

0-2, 2-5, 5-10, 10-20, 20-30 years

In [3]:
# Calculate exact number of days between trading date and maturity
df['maturity_days'] = (df['maturity'] - df['trd_exctn_dt']).dt.days

# Create bins for these maturity days with precise buckets
bins = [0, 730, 1825, 3650, 7300, 10950]  # 0-2, 2-5, 5-10, 10-20, 20-30 years in days
labels = ['0-2 years', '2-5 years', '5-10 years', '10-20 years', '20-30 years']

# Apply binning using cut
df['maturity_bucket'] = pd.cut(df['maturity_days'], bins=bins, labels=labels)


In [4]:
df.head()


,Unnamed: 0,cusip_id,trd_exctn_dt,prclean,prfull,acclast,accpmt,accall,ytm,ytmt,qvolume,dvolume,coupon,mod_dur,convexity,cs_dur,cs,issuer_id,maturity,maturity_days,maturity_bucket
578,578,001765AC0,2002-07-01,84.357102,87.057102,2.700,139.5,142.200,0.112242,0.112242,28000.0,23620.0,9.0,7.117605,77.440707,0.066720,0.060416,001765,2016-09-15,5190,10-20 years
579,579,001765AC0,2002-07-02,85.847800,88.597800,2.750,139.5,142.250,0.109871,0.109871,460000.0,394900.0,9.0,7.175961,78.385893,0.065183,0.058848,001765,2016-09-15,5189,10-20 years
580,580,001765AC0,2002-07-08,85.183698,88.058698,2.875,139.5,142.375,0.110933,0.110933,49000.0,41740.0,9.0,7.134280,77.732856,0.065494,0.059222,001765,2016-09-15,5183,10-20 years
581,581,001765AC0,2002-07-10,86.250001,89.175001,2.925,139.5,142.425,0.109255,0.109255,20000.0,17250.0,9.0,7.174097,78.379230,0.065764,0.059223,001765,2016-09-15,5181,10-20 years
582,582,001765AC0,2002-07-12,84.145802,87.170802,3.025,139.5,142.525,0.112614,0.112614,60000.0,50487.0,9.0,7.073441,76.779550,0.069534,0.062887,001765,2016-09-15,5179,10-20 years


In [5]:
df['maturity_bucket'].value_counts()

maturity_bucket
5-10 years     4699629
2-5 years      4061803
20-30 years    2226897
0-2 years      2226353
10-20 years    1476241
Name: count, dtype: int64

## Yield Curvature Interpolation

Only needed if some dates or yield data is missing for direct comparison. Otherwise, not needed.




In [6]:
from scipy.interpolate import interp1d

# Example of interpolating yields over maturities for a given issuer
def interpolate_yield_curve(df_issuer):
    # Retain original data (which will be merged back later)
    df_original = df_issuer.copy()

    # Aggregate yields by averaging for bonds with the same 'maturity_days'
    df_issuer_agg = df_issuer.groupby('maturity_days', as_index=False).agg({'ytm': 'mean'})

    # Prepare for interpolation
    maturity_days = df_issuer_agg['maturity_days'].values
    yields = df_issuer_agg['ytm'].values

    # Perform interpolation (linear or cubic)
    f = interp1d(maturity_days, yields, kind='cubic', fill_value="extrapolate")
    
    # Generate interpolated yields for any maturity
    interpolated_yields = f(maturity_days)
    
    # Add interpolated yields to aggregated DataFrame
    df_issuer_agg['interpolated_yields'] = interpolated_yields
    
    # Merge the interpolated yields back to the original DataFrame
    df_merged = pd.merge(df_original, df_issuer_agg[['maturity_days', 'interpolated_yields']], on='maturity_days', how='left')

    return df_merged

# Apply interpolation for each issuer
df = df.groupby('issuer_id').apply(interpolate_yield_curve)



/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/750461104.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('issuer_id').apply(interpolate_yield_curve)


In [7]:
df.head()

Unnamed: 0   cusip_id trd_exctn_dt     prclean      prfull  \
issuer_id                                                                 
00037B    0    12594290  00037BAA0   2012-05-03   99.313201   99.313201   
          1    12594291  00037BAA0   2012-05-04  100.056698  100.056698   
          2    12594292  00037BAA0   2012-05-07  100.000703  100.005217   
          3    12594293  00037BAA0   2012-05-08  100.103902  100.112930   
          4    12594294  00037BAA0   2012-05-09  100.153901  100.167442   

              acclast  accpmt    accall       ytm      ytmt      qvolume  \
issuer_id                                                                  
00037B    0  0.000000     0.0  0.000000  0.017681  0.017681  524525000.0   
          1  0.000000     0.0  0.000000  0.016132  0.016132   76926000.0   
          2  0.004514     0.0  0.004514  0.016248  0.016248   18594000.0   
          3  0.009028     0.0  0.009028  0.016033  0.016033    9618000.0   
          4  0.013542     0.0  0.013542  0.015928  0.015928    7270000.0   

                 dvolume  coupon   mod_dur  convexity    cs_dur        cs  \
issuer_id                                                                   
00037B    0  520922458.0   1.625  4.782246  25.767150  0.009873  0.009447   
          1   76969627.0   1.625  4.783981  25.784632  0.008699  0.008306   
          2   18594133.0   1.625  4.780887  25.753498  0.008728  0.008343   
          3    9627990.0   1.625  4.778757  25.732078  0.008701  0.008334   
          4    7281187.0   1.625  4.776304  25.707427  0.008601  0.008233   

            issuer_id   maturity  maturity_days maturity_bucket  \
issuer_id                                                         
00037B    0    00037B 2017-05-08           1831      5-10 years   
          1    00037B 2017-05-08           1830      5-10 years   
          2    00037B 2017-05-08           1827      5-10 years   
          3    00037B 2017-05-08           1826      5-10 years   
          4    00037B 2017-05-08           1825       2-5 years   

             interpolated_yields  
issuer_id                         
00037B    0             0.024628  
          1             0.020142  
          2             0.024612  
          3             0.024486  
          4             0.024794

In [8]:
df.head(900)

Unnamed: 0   cusip_id trd_exctn_dt     prclean      prfull  \
issuer_id                                                                   
00037B    0      12594290  00037BAA0   2012-05-03   99.313201   99.313201   
          1      12594291  00037BAA0   2012-05-04  100.056698  100.056698   
          2      12594292  00037BAA0   2012-05-07  100.000703  100.005217   
          3      12594293  00037BAA0   2012-05-08  100.103902  100.112930   
          4      12594294  00037BAA0   2012-05-09  100.153901  100.167442   
...                   ...        ...          ...         ...         ...   
          895    12595185  00037BAB8   2012-10-04  103.446603  104.644520   
          896    12595186  00037BAB8   2012-10-05  103.095994  104.301897   
          897    12595187  00037BAB8   2012-10-09  103.407402  104.629277   
          898    12595188  00037BAB8   2012-10-10  102.840803  104.070664   
          899    12595189  00037BAB8   2012-10-11  103.250999  104.504819   

                acclast  accpmt    accall       ytm      ytmt      qvolume  \
issuer_id                                                                    
00037B    0    0.000000     0.0  0.000000  0.017681  0.017681  524525000.0   
          1    0.000000     0.0  0.000000  0.016132  0.016132   76926000.0   
          2    0.004514     0.0  0.004514  0.016248  0.016248   18594000.0   
          3    0.009028     0.0  0.009028  0.016033  0.016033    9618000.0   
          4    0.013542     0.0  0.013542  0.015928  0.015928    7270000.0   
...                 ...     ...       ...       ...       ...          ...   
          895  1.197917     0.0  1.197917  0.024689  0.024689    7615000.0   
          896  1.205903     0.0  1.205903  0.025094  0.025094    1555000.0   
          897  1.221875     0.0  1.221875  0.024731  0.024731    2165000.0   
          898  1.229861     0.0  1.229861  0.025388  0.025388   39881000.0   
          899  1.253819     0.0  1.253819  0.024909  0.024909     801000.0   

                   dvolume  coupon   mod_dur  convexity    cs_dur        cs  \
issuer_id                                                                     
00037B    0    520922458.0   1.625  4.782246  25.767150  0.009873  0.009447   
          1     76969627.0   1.625  4.783981  25.784632  0.008699  0.008306   
          2     18594133.0   1.625  4.780887  25.753498  0.008728  0.008343   
          3      9627990.0   1.625  4.778757  25.732078  0.008701  0.008334   
          4      7281187.0   1.625  4.776304  25.707427  0.008601  0.008233   
...                    ...     ...       ...        ...       ...       ...   
          895    7877461.0   2.875  8.271545  78.911499  0.011319  0.008548   
          896    1603142.0   2.875  8.264550  78.802190  0.011238  0.008459   
          897    2238771.0   2.875  8.262868  78.764783  0.010979  0.008219   
          898   41013933.0   2.875  8.253233  78.616794  0.011856  0.009081   
          899     827041.0   2.875  8.250033  78.550792  0.011467  0.008779   

              issuer_id   maturity  maturity_days maturity_bucket  \
issuer_id                                                           
00037B    0      00037B 2017-05-08           1831      5-10 years   
          1      00037B 2017-05-08           1830      5-10 years   
          2      00037B 2017-05-08           1827      5-10 years   
          3      00037B 2017-05-08           1826      5-10 years   
          4      00037B 2017-05-08           1825       2-5 years   
...                 ...        ...            ...             ...   
          895    00037B 2022-05-08           3503      5-10 years   
          896    00037B 2022-05-08           3502      5-10 years   
          897    00037B 2022-05-08           3498      5-10 years   
          898    00037B 2022-05-08           3497      5-10 years   
          899    00037B 2022-05-08           3496      5-10 years   

               interpolated_yields  
issuer_id                          

In [9]:
df = df.reset_index(drop=True)

def sample_size_per_bucket(df_interpolated):
    
    # Group by issuer_id and maturity_bucket, then count the number of bonds in each bucket
    bucket_counts = df_interpolated.groupby(['issuer_id', 'maturity_bucket']).size().reset_index(name='sample_size')
    
    return bucket_counts

# Apply sample size calculation
sample_sizes = sample_size_per_bucket(df)

# Display the sample sizes in each bucket
print(sample_sizes)


/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/2245142395.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_counts = df_interpolated.groupby(['issuer_id', 'maturity_bucket']).size().reset_index(name='sample_size')


     issuer_id maturity_bucket  sample_size
0       00037B       0-2 years          760
1       00037B       2-5 years         1516
2       00037B      5-10 years         1463
3       00037B     10-20 years           60
4       00037B     20-30 years          861
...        ...             ...          ...
4615    98978V       0-2 years         1456
4616    98978V       2-5 years         2285
4617    98978V      5-10 years         4008
4618    98978V     10-20 years           26
4619    98978V     20-30 years         3242

[4620 rows x 3 columns]


In [10]:
sample_sizes.head(50)

,issuer_id,maturity_bucket,sample_size
0,00037B,0-2 years,760
1,00037B,2-5 years,1516
2,00037B,5-10 years,1463
3,00037B,10-20 years,60
4,00037B,20-30 years,861
5,00101J,0-2 years,1296
6,00101J,2-5 years,3777
7,00101J,5-10 years,2970
8,00101J,10-20 years,34
9,00101J,20-30 years,915


Filter buckets with insufficient sample size

In [11]:
# Define the minimum sample size per bucket
min_sample_size = 50

# Filter sample_sizes to keep only those buckets that have at least min_sample_size bonds
filtered_sample_sizes = sample_sizes[sample_sizes['sample_size'] >= min_sample_size]

# Merge filtered sample sizes with the original dataframe to keep only valid rows
df_filtered = pd.merge(df, filtered_sample_sizes[['issuer_id', 'maturity_bucket']], 
                       on=['issuer_id', 'maturity_bucket'], 
                       how='inner')

# Now df_filtered contains only rows with sufficient sample sizes in each maturity bucket
print(f"Number of rows after filtering by sample size: {len(df_filtered)}")


Number of rows after filtering by sample size: 14686044


Create Yield Curve per Bucket

In [49]:
from scipy.interpolate import interp1d

# Function to interpolate the yield curve for each issuer and maturity bucket
def interpolate_yield_curve_per_bucket(df_group):
    # Aggregate yields by averaging for bonds with the same 'maturity_days'
    df_agg = df_group.groupby('maturity_days', as_index=False).agg({'ytm': 'mean'})
    
    # Prepare maturity days and yields for interpolation
    maturity_days = df_agg['maturity_days'].values
    yields = df_agg['ytm'].values
    
    # Perform cubic interpolation with extrapolation if necessary
    f = interp1d(maturity_days, yields, kind='cubic', fill_value="extrapolate")
    
    # Interpolate yields for all maturity days within this group
    df_group['interpolated_yields'] = f(df_group['maturity_days'].values)
    
    return df_group

# Apply interpolation for each issuer and maturity bucket
df_interpolated = df_filtered.groupby(['issuer_id', 'maturity_bucket']).apply(interpolate_yield_curve_per_bucket)

# Now df_interpolated has interpolated yields for each issuer and maturity bucket
df_interpolated.head(20)


/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/1971735829.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_interpolated = df_filtered.groupby(['issuer_id', 'maturity_bucket']).apply(interpolate_yield_curve_per_bucket)
/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/1971735829.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_interpolated = df_filtered.groupby(['issuer_id', 'maturity_bucket']).apply(interpolate_yield_curve_per_bucket)


Unnamed: 0   cusip_id trd_exctn_dt     prclean  \
issuer_id maturity_bucket                                                       
00037B    0-2 years       460    12594750  00037BAA0   2015-05-11  101.086000   
                          461    12594751  00037BAA0   2015-05-12  101.035999   
                          462    12594752  00037BAA0   2015-05-15  100.962001   
                          463    12594753  00037BAA0   2015-05-18  101.138001   
                          464    12594754  00037BAA0   2015-05-20  100.954501   
                          465    12594755  00037BAA0   2015-05-21  100.993000   
                          466    12594756  00037BAA0   2015-05-26  100.913300   
                          467    12594757  00037BAA0   2015-05-28  100.887000   
                          468    12594758  00037BAA0   2015-05-29  101.052901   
                          469    12594759  00037BAA0   2015-06-02  100.828999   
                          470    12594760  00037BAA0   2015-06-05  100.918599   
                          471    12594761  00037BAA0   2015-06-09  100.748300   
                          472    12594762  00037BAA0   2015-06-12  100.875000   
                          473    12594763  00037BAA0   2015-06-15  100.742000   
                          474    12594764  00037BAA0   2015-06-16  100.806999   
                          475    12594765  00037BAA0   2015-06-17  100.815601   
                          476    12594766  00037BAA0   2015-06-18  100.796401   
                          477    12594767  00037BAA0   2015-06-19  100.647101   
                          478    12594768  00037BAA0   2015-06-22  100.593800   
                          479    12594769  00037BAA0   2015-06-30  100.597300   

                                   prfull   acclast  accpmt    accall  \
issuer_id maturity_bucket                                               
00037B    0-2 years       460  101.108569  0.022569   4.875  4.897569   
                          461  101.063083  0.027083   4.875  4.902083   
                          462  101.011654  0.049653   4.875  4.924653   
                          463  101.192167  0.054167   4.875  4.929167   
                          464  101.017695  0.063194   4.875  4.938194   
                          465  101.074250  0.081250   4.875  4.956250   
                          466  101.003578  0.090278   4.875  4.965278   
                          467  100.990820  0.103819   4.875  4.978819   
                          468  101.161234  0.108333   4.875  4.983333   
                          469  100.946361  0.117361   4.875  4.992361   
                          470  101.058530  0.139931   4.875  5.014931   
                          471  100.897258  0.148958   4.875  5.023958   
                          472  101.046528  0.171528   4.875  5.046528   
                          473  100.918042  0.176042   4.875  5.051042   
                          474  100.987555  0.180556   4.875  5.055556   
                          475  101.000670  0.185069   4.875  5.060069   
                          476  100.995012  0.198611   4.875  5.073611   
                          477  100.850226  0.203125   4.875  5.078125   
                          478  100.801439  0.207639   4.875  5.082639   
                          479  100.841050  0.243750   4.875  5.118750   

                                    ytm      ytmt     qvolume     dvolume  \
issuer_id maturity_bucket                                                   
00037B    0-2 years       460  0.010709  0.010709   1500000.0   1516290.0   
                          461  0.010955  0.010955     50000.0     50518.0   
                          462  0.011296  0.011296     15000.0     15144.0   
                          463  0.010389  0.010389     70000.0     70797.0   
                          464  0.011314  0.011314    330000.0    333150.0   
                          465  0.011087  0.011087    792000.0    799865.0   
                         

Signal Generation: Detect Mean-reversion opportunities

In [72]:
import pandas as pd

# Define the function to detect mean-reversion opportunities
def detect_mean_reversion_opportunity(df_interpolated):
    # Ensure 'maturity_bucket' is not part of the index, reset all indexes
    df_interpolated = df_interpolated.reset_index(drop=True)

    # Group by issuer_id and maturity_bucket, calculating historical mean yield per group
    df_interpolated['historical_mean_yield'] = df_interpolated.groupby(['issuer_id', 'maturity_bucket'])['ytm'].transform('mean')

    # Calculate the deviation from the historical mean for each row
    df_interpolated['yield_deviation'] = df_interpolated['ytm'] - df_interpolated['historical_mean_yield']

    # Define a threshold for significant deviation
    deviation_threshold = 0.1

    # Generate signal for mean-reversion opportunity (True/False)
    df_interpolated['signal'] = abs(df_interpolated['yield_deviation']) > deviation_threshold

    return df_interpolated

# Apply the new signal detection
df_with_signals = detect_mean_reversion_opportunity(df_interpolated)


# Filter for rows where there is a significant arbitrage opportunity
arbitrage_opportunities = df_with_signals[df_with_signals['signal'] == True]

# Drop rows with missing numeric values (to prevent aggregation issues)
numeric_columns = ['ytm', 'prclean', 'historical_mean_yield', 'yield_deviation']
arbitrage_opportunities_cleaned = arbitrage_opportunities.dropna(subset=numeric_columns)
arbitrage_opportunities_cleaned = arbitrage_opportunities_cleaned[['cusip_id', 'trd_exctn_dt', 'prclean', 'ytm', 'mod_dur', 'convexity', 'cs', 'issuer_id', 'maturity_bucket', 'yield_deviation']]
arbitrage_opportunities_cleaned

/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/782014974.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_interpolated['historical_mean_yield'] = df_interpolated.groupby(['issuer_id', 'maturity_bucket'])['ytm'].transform('mean')


,cusip_id,trd_exctn_dt,prclean,ytm,mod_dur,convexity,cs,issuer_id,maturity_bucket,yield_deviation
5611,00101JAK2,2020-03-23,90.345200,0.133181,1.368563,2.606239,0.130862,00101J,0-2 years,0.104927
13813,001055AB8,2009-02-24,99.175000,0.126517,0.128013,0.076586,0.119417,001055,0-2 years,0.107927
13823,001055AB8,2009-03-19,99.589200,0.132786,0.057306,0.030153,0.126786,001055,0-2 years,0.114195
39789,00130HAQ8,2008-09-30,96.631600,0.149560,0.596193,0.642343,0.131760,00130H,0-2 years,0.109341
39797,00130HAQ8,2008-10-30,97.000000,0.150726,0.515782,0.515385,0.137126,00130H,0-2 years,0.110507
...,...,...,...,...,...,...,...,...,...,...
14672717,989701AJ6,2008-10-01,53.250000,0.180755,4.788984,29.735482,0.147951,989701,5-10 years,0.103474
14672719,989701AJ6,2008-10-08,54.166701,0.177462,4.793518,29.768940,0.146108,989701,5-10 years,0.100181
14672720,989701AJ6,2008-10-10,54.172701,0.177579,4.782520,29.659002,0.145054,989701,5-10 years,0.100298
14676079,98978VAJ2,2020-03-17,94.625000,0.122874,0.604049,0.653272,0.119874,98978V,0-2 years,0.103844


In [73]:
arbitrage_opportunities_cleaned[arbitrage_opportunities_cleaned['issuer_id']=='00101J']

,cusip_id,trd_exctn_dt,prclean,ytm,mod_dur,convexity,cs,issuer_id,maturity_bucket,yield_deviation
5611,00101JAK2,2020-03-23,90.3452,0.133181,1.368563,2.606239,0.130862,00101J,0-2 years,0.104927


In [70]:
import pandas as pd

# Define separate aggregation functions for numeric and non-numeric columns
numeric_columns = ['prclean', 'ytm', 'mod_dur', 'convexity', 'cs', 'yield_deviation']
non_numeric_columns = ['cusip_id', 'trd_exctn_dt']

# Group and aggregate numeric columns by taking the median
numeric_agg = arbitrage_opportunities_cleaned.groupby(
    ['issuer_id', 'maturity_bucket']
)[numeric_columns].median().reset_index()

# Group and aggregate non-numeric columns by taking the first occurrence
non_numeric_agg = arbitrage_opportunities_cleaned.groupby(
    ['issuer_id', 'maturity_bucket']
)[non_numeric_columns].first().reset_index()

# Merge the two aggregated DataFrames
arbitrage_opportunities_grouped = pd.merge(numeric_agg, non_numeric_agg, on=['issuer_id', 'maturity_bucket'])

# Output the result to verify
arbitrage_opportunities_grouped.head(40)


/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/1841934250.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  numeric_agg = arbitrage_opportunities_cleaned.groupby(
/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/1841934250.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  non_numeric_agg = arbitrage_opportunities_cleaned.groupby(


,issuer_id,maturity_bucket,prclean,ytm,mod_dur,convexity,cs,yield_deviation,cusip_id,trd_exctn_dt
0,00101J,0-2 years,90.345200,0.133181,1.368563,2.606239,0.130862,0.104927,00101JAK2,2020-03-23
1,00101J,2-5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,00101J,5-10 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,00101J,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,00101J,20-30 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,001055,0-2 years,99.382100,0.129652,0.092660,0.053370,0.123102,0.111061,001055AB8,2009-02-24
6,001055,2-5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7,001055,5-10 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8,001055,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
9,001055,20-30 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [82]:
# Define a function to count rows with NaN for each group
def drop_issuer_with_nans_condition(group):
    # Count the number of rows with NaN values in each group
    nan_count = group.isna().sum(axis=1).gt(0).sum()  # Rows with any NaN values
    if nan_count > 2:  # If more than 2 rows in the group have NaN values
        return None  # Exclude the group
    else:
        return group

# Apply the function to each group
arbitrage_opportunities_cleaned_final = arbitrage_opportunities_grouped.groupby('issuer_id').apply(drop_issuer_with_nans_condition)

# Drop the None rows (i.e., groups that were excluded)
arbitrage_opportunities_cleaned_final = arbitrage_opportunities_cleaned_final.dropna(how='all').reset_index(drop=True)

# Output the result to verify
arbitrage_opportunities_cleaned_final.head(40)


/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/39602018.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  arbitrage_opportunities_cleaned_final = arbitrage_opportunities_grouped.groupby('issuer_id').apply(drop_issuer_with_nans_condition)


,issuer_id,maturity_bucket,prclean,ytm,mod_dur,convexity,cs,yield_deviation,cusip_id,trd_exctn_dt
0,00130H,0-2 years,97.000000,0.151004,0.475997,0.456945,0.139024,0.110784,00130HAQ8,2008-09-30
1,00130H,2-5 years,88.000000,0.155917,1.804724,4.344236,0.146628,0.106854,00130HAU9,2008-11-24
2,00130H,5-10 years,64.146352,0.176978,4.526247,30.374612,0.140683,0.114925,00130HAQ8,2002-07-01
3,00130H,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,00130H,20-30 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,026351,0-2 years,90.286200,0.149655,1.322222,2.409292,0.142311,0.100829,026351BC9,2008-11-13
6,026351,2-5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7,026351,5-10 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8,026351,10-20 years,45.086350,0.172476,5.898582,63.238327,0.135452,0.107965,026351AU0,2008-11-24
9,026351,20-30 years,46.833301,0.186583,5.525327,59.132569,0.141156,0.123010,026351BB1,2009-08-05


In [92]:
arbitrage_opportunities=arbitrage_opportunities_cleaned_final

In [93]:
arbitrage_opportunities

,issuer_id,maturity_bucket,prclean,ytm,mod_dur,convexity,cs,yield_deviation,cusip_id,trd_exctn_dt
0,00130H,0-2 years,97.000000,0.151004,0.475997,0.456945,0.139024,0.110784,00130HAQ8,2008-09-30
1,00130H,2-5 years,88.000000,0.155917,1.804724,4.344236,0.146628,0.106854,00130HAU9,2008-11-24
2,00130H,5-10 years,64.146352,0.176978,4.526247,30.374612,0.140683,0.114925,00130HAQ8,2002-07-01
3,00130H,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,00130H,20-30 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
500,989701,0-2 years,99.875000,0.146411,0.012942,0.006197,0.141711,0.125961,989701AM9,2015-11-09
501,989701,2-5 years,67.270598,0.154327,3.892895,18.626203,0.131396,0.106718,989701AL1,2009-05-20
502,989701,5-10 years,54.166701,0.177579,4.788984,29.735482,0.146108,0.100298,989701AJ6,2008-10-01
503,989701,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [96]:
# Group by 'issuer_id' and sort each group by 'yield_deviation'
arbitrage_opportunities = arbitrage_opportunities.groupby('issuer_id', group_keys=False).apply(
    lambda x: x.sort_values(by='yield_deviation', ascending=False)
)

arbitrage_opportunities.head(20)


/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/870187511.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  arbitrage_opportunities = arbitrage_opportunities.groupby('issuer_id', group_keys=False).apply(


,issuer_id,maturity_bucket,prclean,ytm,mod_dur,convexity,cs,yield_deviation,cusip_id,trd_exctn_dt
2,00130H,5-10 years,64.146352,0.176978,4.526247,30.374612,0.140683,0.114925,00130HAQ8,2002-07-01
0,00130H,0-2 years,97.000000,0.151004,0.475997,0.456945,0.139024,0.110784,00130HAQ8,2008-09-30
1,00130H,2-5 years,88.000000,0.155917,1.804724,4.344236,0.146628,0.106854,00130HAU9,2008-11-24
3,00130H,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,00130H,20-30 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
9,026351,20-30 years,46.833301,0.186583,5.525327,59.132569,0.141156,0.123010,026351BB1,2009-08-05
8,026351,10-20 years,45.086350,0.172476,5.898582,63.238327,0.135452,0.107965,026351AU0,2008-11-24
5,026351,0-2 years,90.286200,0.149655,1.322222,2.409292,0.142311,0.100829,026351BC9,2008-11-13
6,026351,2-5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7,026351,5-10 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [95]:
arbitrage_opportunities

,issuer_id,maturity_bucket,prclean,ytm,mod_dur,convexity,cs,yield_deviation,cusip_id,trd_exctn_dt
2,00130H,5-10 years,64.146352,0.176978,4.526247,30.374612,0.140683,0.114925,00130HAQ8,2002-07-01
0,00130H,0-2 years,97.000000,0.151004,0.475997,0.456945,0.139024,0.110784,00130HAQ8,2008-09-30
1,00130H,2-5 years,88.000000,0.155917,1.804724,4.344236,0.146628,0.106854,00130HAU9,2008-11-24
3,00130H,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,00130H,20-30 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
500,989701,0-2 years,99.875000,0.146411,0.012942,0.006197,0.141711,0.125961,989701AM9,2015-11-09
501,989701,2-5 years,67.270598,0.154327,3.892895,18.626203,0.131396,0.106718,989701AL1,2009-05-20
502,989701,5-10 years,54.166701,0.177579,4.788984,29.735482,0.146108,0.100298,989701AJ6,2008-10-01
503,989701,10-20 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


Create trading signals

In [40]:
# Define buy/sell signals
arbitrage_opportunities['action'] = arbitrage_opportunities['yield_deviation'].apply(lambda x: 'Buy' if x > 0 else 'Sell')

arbitrage_opportunities[['issuer_id', 'maturity_bucket', 'yield_deviation', 'trd_exctn_dt', 'prclean', 'action']]
#arbitrage_opportunities

,issuer_id,maturity_bucket,yield_deviation,trd_exctn_dt,prclean,action
7873751,438516,0-2 years,0.174394,2006-10-25,99.840000,Buy
4243420,235811,0-2 years,0.174101,2006-07-21,81.844199,Buy
7412783,382550,0-2 years,0.172960,2006-11-15,99.500000,Buy
7725397,428040,0-2 years,0.172490,2006-06-16,96.000000,Buy
12247002,812404,0-2 years,0.172456,2007-06-05,99.722200,Buy
...,...,...,...,...,...,...
3523233,184502,2-5 years,0.100001,2008-03-11,77.237601,Buy
6689706,37042G,2-5 years,0.100001,2005-11-29,82.086577,Buy
8035764,442488,5-10 years,0.100000,2008-02-20,68.968001,Buy
4631344,25470X,2-5 years,0.100000,2022-07-08,85.209601,Buy


In [41]:
arbitrage_opportunities[arbitrage_opportunities['issuer_id']=='438516']

,Unnamed: 0,cusip_id,trd_exctn_dt,prclean,prfull,acclast,accpmt,accall,ytm,ytmt,qvolume,dvolume,coupon,mod_dur,convexity,cs_dur,cs,issuer_id,maturity,maturity_days,maturity_bucket,interpolated_yields,historical_mean_yield,yield_deviation,signal,action,position_size,realized_profit
7873751,2931707,438516AM8,2006-10-25,99.840000,102.345556,2.505556,23.076736,25.582292,0.199948,0.199948,10000.0,9984.0,5.125,0.010101,0.004694,0.149148,0.149148,438516,2006-11-01,7,0-2 years,0.057386,0.025555,0.174394,True,Buy,100,0.000000
7872541,37164,438516AJ5,2009-06-10,98.065934,98.065934,0.000000,0.000000,0.000000,0.147815,0.153277,45000.0,44100.0,0.000,0.123423,0.122762,0.147777,0.147777,438516,2009-08-01,52,0-2 years,0.035372,0.025555,0.122260,True,Buy,100,0.000000
7872455,37078,438516AJ5,2008-10-23,90.328986,90.328986,0.000000,0.000000,0.000000,0.141870,0.146902,60000.0,54015.0,0.000,0.671413,1.038790,0.130802,0.130802,438516,2009-08-01,282,0-2 years,0.033269,0.025555,0.116316,True,Buy,100,773.694848
7872472,37095,438516AJ5,2008-11-28,92.188053,92.188053,0.000000,0.000000,0.000000,0.129193,0.133366,30000.0,27588.0,0.000,0.592852,0.876496,0.124366,0.124366,438516,2009-08-01,246,0-2 years,0.029217,0.025555,0.103638,True,Buy,100,587.788100


In [27]:
# position size

def allocate_position(yield_deviation):
    if abs(yield_deviation) > 0.02:  # Large deviation, allocate more capital
        return 100  # $1,000,000
    else:
        return 50  # $500,000

arbitrage_opportunities['position_size'] = arbitrage_opportunities['yield_deviation'].apply(allocate_position)


Simulation

In [36]:
def calculate_realized_profit_with_reversion(df, reversion_threshold=0.2):
    df['realized_profit'] = 0  # Initialize profit column

    for index, row in df.iterrows():
        # Entry details
        entry_price = row['prclean']
        entry_date = row['trd_exctn_dt']
        position_size = row['position_size']
        action = row['action']

        # Historical data for the bond (future prices after the entry date)
        df_future = df[(df['cusip_id'] == row['cusip_id']) & (df['trd_exctn_dt'] > entry_date)]

        for _, future_row in df_future.iterrows():
            future_price = future_row['prclean']
            yield_deviation = future_row['yield_deviation']
            
            # If yield deviation reverts to a threshold close to zero, exit trade
            if abs(yield_deviation) < reversion_threshold:
                if action == 'Buy':
                    df.at[index, 'realized_profit'] = position_size * (future_price - entry_price)
                elif action == 'Sell':
                    df.at[index, 'realized_profit'] = position_size * (entry_price - future_price)
                break  # Exit the trade once the reversion occurs
    
    return df

# Apply the profit calculation using the existing 'arbitrage_opportunities' dataframe
arbitrage_opportunities = calculate_realized_profit_with_reversion(arbitrage_opportunities)

# Display the results
arbitrage_opportunities[['cusip_id', 'trd_exctn_dt', 'ytm', 'historical_mean_yield', 'position_size', 'realized_profit']]


/var/folders/x5/pnjs0yvx6r96_n17gffyc9rh0000gn/T/ipykernel_1776/2921411772.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '117.47009277888765' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'realized_profit'] = position_size * (future_price - entry_price)


,cusip_id,trd_exctn_dt,ytm,historical_mean_yield,position_size,realized_profit
7873751,438516AM8,2006-10-25,0.199948,0.025555,100,0.000000
4243420,235811AH9,2006-07-21,0.199656,0.025555,100,117.470093
7412783,382550AC5,2006-11-15,0.198515,0.025555,100,0.000000
7725397,428040BT5,2006-06-16,0.198045,0.025555,100,256.690003
12247002,812404AV3,2007-06-05,0.198011,0.025555,100,0.000000
...,...,...,...,...,...,...
3523233,184502AR3,2008-03-11,0.134348,0.034347,100,-347.940067
6689706,37042G4R4,2005-11-29,0.134348,0.034347,100,1739.977489
8035764,442488AQ5,2008-02-20,0.145022,0.045022,100,-699.180258
4631344,25470XAW5,2022-07-08,0.134347,0.034347,100,0.000000


In [30]:
arbitrage_opportunities[['cusip_id', 'trd_exctn_dt', 'ytm', 'historical_mean_yield', 'position_size', 'realized_profit']].head(100)

,cusip_id,trd_exctn_dt,ytm,historical_mean_yield,position_size,realized_profit
7873751,438516AM8,2006-10-25,0.199948,0.025555,100,0.000000
4243420,235811AH9,2006-07-21,0.199656,0.025555,100,117.470093
7412783,382550AC5,2006-11-15,0.198515,0.025555,100,0.000000
7725397,428040BT5,2006-06-16,0.198045,0.025555,100,256.690003
12247002,812404AV3,2007-06-05,0.198011,0.025555,100,0.000000
...,...,...,...,...,...,...
4243425,235811AH9,2006-08-01,0.184468,0.025555,100,-83.309931
2810847,131347AD8,2005-08-09,0.184372,0.025555,100,5.149991
3158424,16117PAE0,2006-05-24,0.193138,0.034347,100,-85.810013
2811086,131347AL0,2005-07-29,0.184281,0.025555,100,59.189968


In [35]:
profits_by_issuer = arbitrage_opportunities.groupby('issuer_id').agg(
    total_realized_profit=pd.NamedAgg(column='realized_profit', aggfunc='sum'),
    total_position_size=pd.NamedAgg(column='position_size', aggfunc='sum'),
    average_yield=pd.NamedAgg(column='ytm', aggfunc='mean'),
    trade_count=pd.NamedAgg(column='cusip_id', aggfunc='count')  # Count number of trades for each issuer
).reset_index()

# Display the profits by issuer
profits_by_issuer.head(40)

,issuer_id,total_realized_profit,total_position_size,average_yield,trade_count
0,00101J,0.000000,100,0.133181,1
1,001055,41.419997,200,0.129652,2
2,00130H,120275.217305,14400,0.161423,144
3,001546,11737.770988,26900,0.150340,269
4,00164V,-1681.589401,700,0.148867,7
5,001669,-3867.788557,3800,0.151771,38
6,001765,-20479.935935,75000,0.165767,750
7,00184A,0.000000,200,0.136239,2
8,001957,0.000000,400,0.144419,4
9,00206R,0.000000,100,0.132746,1


In [31]:
arbitrage_opportunities['realized_profit'].sum()

np.float64(-2973010.7219155463)

# Stress test

In [32]:
import pandas as pd
import numpy as np

# Function to calculate price change using duration and convexity
def price_change(duration, convexity, yield_change):
    """
    Estimate bond price change using duration and convexity approximation.
    """
    return -(duration * yield_change) + (0.5 * convexity * (yield_change ** 2))

# Function to adjust yield_deviation based on stress test shifts
def adjust_yield_deviation(yield_deviation, yield_change):
    """
    Adjust yield deviation based on simulated yield curve changes.
    """
    return yield_deviation + yield_change

# Function to run stress test for yield curve movements
def yield_curve_stress_test(df, steepening_shift=0.01, flattening_shift=0.01, inversion_shift=0.01):
    """
    Simulates three types of yield curve movements: steepening, flattening, and inversion.
    Args:
    df: DataFrame containing bond information with 'cusip_id', 'mod_dur', 'convexity', 'maturity_days', 'yield_deviation', 'prclean'
    steepening_shift: Simulated increase in long-term yields for steepening
    flattening_shift: Simulated increase in short-term yields for flattening
    inversion_shift: Simulated increase in short-term yields and decrease in long-term yields for inversion
    
    Returns:
    A DataFrame showing the impact of each scenario on bond prices, yield deviations, portfolio value, and percentage change from the baseline.
    """
    
    # Copy the original DataFrame to avoid modifying it directly
    df_stress_test = df.copy()
    
    # Calculate baseline portfolio value using current clean prices
    baseline_portfolio_value = df['prclean'].sum()

    # Define stress test scenarios
    scenarios = {
        'steepening': {'short_term_shift': 0, 'long_term_shift': steepening_shift},
        'flattening': {'short_term_shift': flattening_shift, 'long_term_shift': 0},
        'inversion': {'short_term_shift': inversion_shift, 'long_term_shift': -inversion_shift}
    }
    
    # List to store results for each scenario
    results = []

    # Iterate through each scenario
    for scenario_name, shifts in scenarios.items():
        # Apply yield changes based on maturity buckets (short-term vs long-term)
        df_stress_test['yield_change'] = np.where(df_stress_test['maturity_days'] <= 730,  # Short-term (< 2 years)
                                                  shifts['short_term_shift'],
                                                  shifts['long_term_shift'])  # Long-term
        
        # Adjust the yield deviations based on the stress test
        df_stress_test['adjusted_yield_deviation'] = adjust_yield_deviation(df_stress_test['yield_deviation'],
                                                                           df_stress_test['yield_change'])
        
        # Calculate price changes based on modified duration, convexity, and new yield_deviation
        df_stress_test['price_change'] = price_change(df_stress_test['mod_dur'],
                                                      df_stress_test['convexity'],
                                                      df_stress_test['adjusted_yield_deviation'])
        
        # Recalculate new bond prices (starting with price = prclean for each bond)
        df_stress_test['new_price'] = df_stress_test['prclean'] + df_stress_test['price_change']
        
        # Calculate the total portfolio value under this scenario
        portfolio_value = df_stress_test['new_price'].sum()
        
        # Calculate percentage change from baseline
        pct_change_from_baseline = ((portfolio_value - baseline_portfolio_value) / baseline_portfolio_value) * 100
        
        # Store the result
        results.append({
            'scenario': scenario_name,
            'portfolio_value': portfolio_value,
            'percentage_change_from_baseline': pct_change_from_baseline,
            'average_price_change': df_stress_test['price_change'].mean(),
            'max_price_change': df_stress_test['price_change'].max(),
            'min_price_change': df_stress_test['price_change'].min()
        })
    
    # Add the baseline (current portfolio value) to the results
    results.append({
        'scenario': 'baseline',
        'portfolio_value': baseline_portfolio_value,
        'percentage_change_from_baseline': 0.0,  # No change from baseline
        'average_price_change': 0,  # Baseline has no change in prices
        'max_price_change': 0,
        'min_price_change': 0
    })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Now, use the arbitrage_opportunities DataFrame for stress testing
# Ensure that arbitrage_opportunities contains necessary columns: 'mod_dur', 'convexity', 'maturity_days', 'prclean', 'yield_deviation'
# Apply the stress test simulation
stress_test_results = yield_curve_stress_test(arbitrage_opportunities)

# Display the results
stress_test_results


,scenario,portfolio_value,percentage_change_from_baseline,average_price_change,max_price_change,min_price_change
0,steepening,4.763482e+06,-0.374042,-0.271242,0.372292,-0.444313
1,flattening,4.763895e+06,-0.365411,-0.264983,0.167432,-0.451667
2,inversion,4.764535e+06,-0.352016,-0.255269,-0.000289,-0.451553
3,baseline,4.781366e+06,0.000000,0.000000,0.000000,0.000000


## For a 100bp (1%) change in yield curve adjustment, 

Each of the scenarios have an adverse movement of -0.36% to total portfolio value

If the trade is leveraged x10:
1% adjustment in curvature -> 3.6% adverse movement

if leveraged x50:
1% adjustment in curvature -> 18% adverse movement


## Estimating Probability of loss

assuming curvature deviation of 1standard deviation is 1% => 100-86=14% chance of 18% adverse movement in 1 trading year

2sd is 2% => 100-95=5% chance of 36% adverse movement in 1 trading year